<a href="https://colab.research.google.com/github/ArthurAJO/aerospace_engineering_ai_activities/blob/main/secondList/Function_Approximation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nome: Arthur Augusto J de Oliveira\
Matrícula: 2016021645

**Link do notebook deste Google Colab: https://colab.research.google.com/drive/13PH7yPRQ3BuRWBaqWcWdyT5wbaHsplXd#scrollTo=TV6ddBVZ0_Kc**

## 3. Aproximação de funções

Com o intuito de observar a capacidade do MLP de aproximar qualquer
função contínua, será realizada a regressão de um intervalo da função seno
cardinal (sinc(x)) com backpropagation. A função de ativação da camada
de saída deve ser linear, e o número de neurônios na camada escondida deve
ser modificado com o intuito de se encontrar a melhor aproximação para a
função.
Devem ser realizadas duas execuções do modelo, sendo uma com um
número de amostras de treinamento igual a 45 e a outra 200, sendo x
amostrado entre −2π e 2π. O valor da função sinc(x) deve ser acrescido
a um ruído uniforme, amostrado no intervalo [−0.1, 0.1]. O conjunto de teste
deve ser composto de valores de x entre [−2π, 2π], obtidos com passo δ = 0.01
e y = sinc(x), sendo:

sinc(x) = sin(x)/x

Para cada execução do modelo, modificando-se os dados de treinamento,
deve ser calculado o erro quadrático médio (MSE) e um gráfico comparando
a saída da funação aproximada e os valores esperados de y.

## Inicialização

Imports e comunicação com o repositório do GitHub que contém o banco de dados para os programas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle

!git clone https://github.com/ArthurAJO/aerospace_engineering_ai_activities.git